# Batch Image Downloader for jav.ink of yamachan01.com

In [1]:
from user_agent import generate_user_agent
from bs4 import BeautifulSoup
from random import randint
from pathlib import Path
import urllib.parse
import requests
import shutil
import os

In [2]:
##  https://yamachan01.com/?tag=s-cute
##  https://yamachan01.com/blog-entry-2324.html
_category = "s-cute"
_url = "https://yamachan01.com/?tag="+_category

_outputPath = "../Dataset/Photos/Adult/yamachan01.com/"+_category+"/"

In [9]:
def downloadAlbum(url):

    if (len(url) < 10):
        return

    ##  If folder already exists, means already downloaded, skip it
    path = Path(url)
    pathName = urllib.parse.unquote(path.name)
    filePath = os.path.join(_outputPath, pathName)
    exist = os.path.exists(filePath)

    ##  Create path
    if (exist == False):
        os.makedirs(filePath)

    userAgent = generate_user_agent()
    headers = {"user-agent":userAgent, "referer":_url}
    httpRequest = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(httpRequest.text, "html.parser")
    print(httpRequest.url)

    objectList = soup.findAll("a", attrs={"target":"_blank"})
    if (objectList is None):
        return

    for imageObject in objectList:

        imageURL = imageObject.get("href")
        if (imageURL is None):
            print("### Image URL not found...")
            print(soup)
            finish = True
            return
        
        ##  Check if the link I want
        if "yamachan01" not in imageURL:
            continue

        ##  Download image
        imageURL = imageURL.rstrip()
        imageFilename = imageURL.split("/")[-1]
        
        array = imageFilename.split(".jpg")
        imageFilename = array[0]+".jpg"
        
        ##  Check if file already exists
        path = os.path.join(filePath, imageFilename)
        if (os.path.isfile(path) == True):
            continue

        #print("Downloading "+imageURL+"...")
        imageURL = imageURL+"?v="+str(randint(1, 100))
        httpRequest = requests.get(imageURL, headers=headers, stream=True)
        if (httpRequest.status_code != 200):
            print("### Unable to download image...\n")
            continue

        ##  Set true to prevent image size zero
        httpRequest.raw.decode_content = True

        ##  Save image
        with open(path, "wb") as file:
            shutil.copyfileobj(httpRequest.raw, file)


In [ ]:
finish = False
albumURL = _url
userAgent = generate_user_agent()
for i in range(1):

    page = i+1
    print("Album page #"+str(page)+"...")

    url = albumURL
    if (page > 1):
        url = albumURL+"page/"+str(page)+"/"
    
    headers = {"user-agent":userAgent, "referer":_url}
    httpRequest = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(httpRequest.text, "html.parser")

    albumList = soup.findAll("a", attrs={"rel":"nofollow"})
    if (albumList is None):
        finish = True
        break

    finish = True
    for linkObject in albumList:

        ##  Check if it is Graphis album
        url = linkObject.get("href")
        
        if (url.startswith("https://yamachan01.com") == False):
            continue
        
        ##  Yes, Graphis album
        #print(url)
        downloadAlbum(url)
        finish = False

    if (finish == True):
        break

print("Finish all!")

Album page #1...
https://yamachan01.com/blog-entry-2324.html
https://yamachan01.com/blog-entry-2224.html
https://yamachan01.com/blog-entry-2117.html
https://yamachan01.com/blog-entry-2016.html
https://yamachan01.com/blog-entry-1685.html
https://yamachan01.com/blog-entry-1622.html
https://yamachan01.com/blog-entry-997.html
